# Basic Configs

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils import data

import numpy as np
import matplotlib.pyplot as plt

## Global Variables

In [3]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
cpu = torch.device("cpu")
batch_size = 128
num_workers = 4

## Load Repo
- Rename the folder to `project`

In [4]:
!git clone https://github.com/effie-0/IDL-Project.git

Cloning into 'IDL-Project'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 185 (delta 83), reused 95 (delta 32), pack-reused 0
Receiving objects: 100% (185/185), 1.59 MiB | 1.78 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [5]:
!mv IDL-Project project

# Load Functions

In [6]:
from project.summarize.load_data import AdvDataset, get_data

## Load Dataset

In [16]:
trainset, trainloader, testset, testloader = get_data(batch_size)

Files already downloaded and verified
Files already downloaded and verified


## Load Models

In [8]:
from project.summarize.classifier import ResidualBlock, ResNet18, cifar_resnet20, ResNet50, VGG

In [9]:
resnet18 = ResNet18(ResidualBlock)
resnet20 = cifar_resnet20('cifar10')  # With pretrained model
resnet50 = ResNet50()
vgg16 = VGG('VGG16')

Downloading: "https://github.com/chenyaofo/CIFAR-pretrained-models/releases/download/resnet/cifar10-resnet20-30abc31d.pth" to /root/.cache/torch/hub/checkpoints/cifar10-resnet20-30abc31d.pth


### Load Trained Models

In [10]:
vgg16_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/vgg16.pth"
vgg16.load_state_dict(torch.load(vgg16_path)['model_state_dict'])

resnet18_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/ResNet18.pth"
resnet18.load_state_dict(torch.load(resnet18_path)['model_state_dict'])

resnet50_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/Copy of ResNet50_49.pth"
resnet50.load_state_dict(torch.load(resnet50_path)['model_state_dict'])

<All keys matched successfully>

In [11]:
models = {
    'resnet18': resnet18,
    'resnet20': resnet20,
    'resnet50': resnet50,
    'vgg16': vgg16,
}

# Generate the Adversarial Samples

In [13]:
from project.summarize.generate_attack import generate_fgsm_attack

In [14]:
criterion = nn.CrossEntropyLoss()

## Evaluate the models

In [21]:
eps = 0.0

for model_name, model in models.items():
    model.to(device)
    model.eval()
    acc, adv_ex, adv_label = generate_fgsm_attack(model, criterion, testloader, testset, eps, device)
    loss, acc = evaluate(model, testloader, criterion, device)
    print('eval acc = ', acc)
    model.to(cpu)
    torch.cuda.empty_cache()

Epsilon: 0.0	Test Accuracy = 9054 / 10000 = 0.9054
eval acc =  90.53999999999999
Epsilon: 0.0	Test Accuracy = 9165 / 10000 = 0.9165
eval acc =  91.64999999999999
Epsilon: 0.0	Test Accuracy = 9081 / 10000 = 0.9081
eval acc =  90.81
Epsilon: 0.0	Test Accuracy = 9231 / 10000 = 0.9231
eval acc =  92.31


In [17]:
adv_samples = {}
eps = 0.05

for model_name, model in models.items():
    model.to(device)
    model.eval()
    acc, adv_ex, adv_label = generate_fgsm_attack(model, criterion, testloader, testset, eps, device)
    model.to(cpu)
    torch.cuda.empty_cache()
    dataset = AdvDataset(adv_ex, adv_label)
    loader = data.DataLoader(dataset, batch_size=batch_size, 
                             shuffle=False,
                             num_workers=num_workers)
    adv_samples[model_name] = (dataset, loader)

Epsilon: 0.05	Test Accuracy = 2244 / 10000 = 0.2244
Epsilon: 0.05	Test Accuracy = 2545 / 10000 = 0.2545
Epsilon: 0.05	Test Accuracy = 2477 / 10000 = 0.2477
Epsilon: 0.05	Test Accuracy = 2014 / 10000 = 0.2014


# 4x4 Matrix

In [18]:
from project.summarize.normal_train import evaluate

In [20]:
matrix = {}
for model_name, model in models.items():
    line = {}
    matrix[model_name] = line
    model.to(device)
    for adv_name, (dataset, loader) in adv_samples.items():
        print('classifier: ', model_name, ';  adversarial data comes from: ', adv_name)
        loss, acc = evaluate(model, loader, criterion, device)
        line[adv_name] = acc
        print('acc = ', acc)
    model.to(cpu)
    torch.cuda.empty_cache()

classifier:  resnet18 ;  adversarial data comes from:  resnet18
acc =  22.439999999999998
classifier:  resnet18 ;  adversarial data comes from:  resnet20
acc =  82.73
classifier:  resnet18 ;  adversarial data comes from:  resnet50
acc =  76.13
classifier:  resnet18 ;  adversarial data comes from:  vgg16
acc =  78.82000000000001
classifier:  resnet20 ;  adversarial data comes from:  resnet18
acc =  71.84
classifier:  resnet20 ;  adversarial data comes from:  resnet20
acc =  25.45
classifier:  resnet20 ;  adversarial data comes from:  resnet50
acc =  65.53
classifier:  resnet20 ;  adversarial data comes from:  vgg16
acc =  71.77
classifier:  resnet50 ;  adversarial data comes from:  resnet18
acc =  73.64
classifier:  resnet50 ;  adversarial data comes from:  resnet20
acc =  76.69
classifier:  resnet50 ;  adversarial data comes from:  resnet50
acc =  24.77
classifier:  resnet50 ;  adversarial data comes from:  vgg16
acc =  76.29
classifier:  vgg16 ;  adversarial data comes from:  resnet18